In [19]:
import summary as msy
import numpy as np
import pandas as pd
import importlib
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
importlib.reload(msy)

Using device: cpu


[nltk_data] Downloading package punkt to /Users/lwk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/lwk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<module 'summary' from '/Users/lwk/Documents/web_science/web_science/summary.py'>

### load dataset

In [21]:
df_1, df_2 = msy.ini('Software_5.json', 'meta_Software.json')

# Part 1

In [22]:
preprocess = msy.Preprocess(df_1, df_2)

In [23]:
training_des = preprocess.training_data.groupby(['reviewerID']).count()
training_des.overall.describe()

count    1824.000000
mean        5.576206
std         3.488828
min         1.000000
25%         4.000000
50%         5.000000
75%         6.000000
max        51.000000
Name: overall, dtype: float64

In [24]:
test_des = preprocess.test_data.groupby(['reviewerID']).count()
test_des.overall.describe()

count    1711.0
mean        1.0
std         0.0
min         1.0
25%         1.0
50%         1.0
75%         1.0
max         1.0
Name: overall, dtype: float64

In [25]:
trainingit_des = preprocess.training_data.groupby(['asin']).count()
trainingit_des.overall.describe()

count    800.000000
mean      12.713750
std       16.234801
min        1.000000
25%        5.000000
50%        9.500000
75%       17.000000
max      226.000000
Name: overall, dtype: float64

In [26]:
trainingit_head = preprocess.training_data.groupby(['asin']).size().reset_index(name='count') 
trainingit_head['mean rate'] = list(preprocess.training_data.groupby(['asin'])['overall'].mean())
print(trainingit_head.sort_values(['count'], ascending=False)\
                            .head(5))
print(trainingit_head.sort_values(['mean rate'], ascending=False)\
                            .head(5))


           asin  count  mean rate
50   B0001FS9NE    226   4.929204
18   B000050ZRE    226   4.929204
41   B0000AZJY6    224   4.928571
560  B00EZPXYP4     91   4.384615
569  B00F8K9MZQ     66   4.318182
           asin  count  mean rate
214  B0015CA9VW      1        5.0
217  B0015T5Q86      3        5.0
213  B0014ZQMXO      2        5.0
206  B0014A2LJ8      1        5.0
191  B0013MCZRK      2        5.0


# Part 2

In [27]:
coll_recommend = msy.Collaborativefiltering_recommendation(reuse=preprocess)

### 3-fold cross-validation

In [28]:
gs_knn = coll_recommend.gridsearch('knn')
gs_svd = coll_recommend.gridsearch('svd')

Computing the msd similarity matrix...
Computing the msd similarity matrix...
Computing the msd similarity matrix...
Computing the cosine similarity matrix...
Computing the cosine similarity matrix...
Computing the cosine similarity matrix...
Computing the pearson similarity matrix...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Computing the pearson similarity matrix...
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Done computing similarity matrix.
Estimating biases using als...
Done computing similarity matrix.
Computing the msd similarity matrix...
Computing the pearson_baseline similarity matrix...
Computing the msd similarity matri

In [29]:
print(gs_knn.best_score)
print(gs_knn.best_params['rmse'])
print(gs_svd.best_score)
print(gs_svd.best_params['rmse'])

{'rmse': 1.2140536187070865}
{'k': 5, 'sim_options': {'name': 'pearson', 'user_based': True}}
{'rmse': 1.1389858056460782}
{'n_epochs': 800, 'n_factors': 50, 'random_state': 0}


In [30]:
knn_est, knn_model = coll_recommend.knn(
    k=5, sim_options={'name': 'pearson', 'user_based': True})
svd_est, svd_model = coll_recommend.svd(n_epochs=800, n_factors=50, random_state=0)


Computing the pearson similarity matrix...
Done computing similarity matrix.


## Part 3

In [31]:
knn_evalue = msy.Evaluation(knn_est, reuse=preprocess)
svd_evalue = msy.Evaluation(svd_est, reuse=preprocess)

In [32]:
print('knn mpk+ {:.6f}'.format(knn_evalue.mean_k(5,'p_k')))
print('knn mapk+ {:.6f}'.format(knn_evalue.mean_k(5,'ap_k')))
print('knn mrrk+ {:.6f}'.format(knn_evalue.mean_k(5,'rr_k')))
print('knn hrk+ {:.6f}'.format(knn_evalue.mean_k(5,'hr_k')))

knn mpk+ 0.003741
knn mapk+ 0.008816
knn mrrk+ 0.008816
knn hrk+ 0.018703


In [33]:
print('svd mpk+ {:.6f}'.format(svd_evalue.mean_k(5,'p_k')))
print('svd mapk+ {:.6f}'.format(svd_evalue.mean_k(5,'ap_k')))
print('svd mrrk+ {:.6f}'.format(svd_evalue.mean_k(5,'rr_k')))
print('svd hrk+ {:.6f}'.format(svd_evalue.mean_k(5,'hr_k')))

svd mpk+ 0.001870
svd mapk+ 0.003078
svd mrrk+ 0.003078
svd hrk+ 0.009351


In [34]:
print('knn rmse+ {}'.format(knn_evalue.rmse()))
print('svd rmse+ {}'.format(svd_evalue.rmse()))

RMSE: 0.9793
knn rmse+ 0.9793299211533906
RMSE: 0.7191
svd rmse+ 0.7190771856550636


In [35]:

first_uid = coll_recommend.surprice_training.build_full_trainset().\
    to_inner_uid(list(coll_recommend.test_data.sort_values(
    by='unixReviewTime')['reviewerID'])[0])
last_uid = coll_recommend.surprice_training.build_full_trainset().\
    to_inner_uid(list(coll_recommend.test_data.sort_values(
    by='unixReviewTime')['reviewerID'])[-1])
nnfor_fisrt = [list(coll_recommend.test_data.sort_values(
    by='unixReviewTime')['reviewerID'])[0]]+[coll_recommend.surprice_training.build_full_trainset().\
    to_raw_uid(i) for i in knn_model.get_neighbors(first_uid, 10)]
nnfor_last = [list(coll_recommend.test_data.sort_values(
    by='unixReviewTime')['reviewerID'])[-1]]+[coll_recommend.surprice_training.build_full_trainset().\
    to_raw_uid(i) for i in knn_model.get_neighbors(last_uid, 10)]


In [36]:
first_user_nn10_matrix = coll_recommend.customer_ui_matrix(user_list=nnfor_fisrt)
last_user_nn10_matrix = coll_recommend.customer_ui_matrix(user_list=nnfor_last)

In [37]:
print(first_user_nn10_matrix)

asin            B005S4Y65I  B009CCVMO0  B00F8K9KZS  B00BR082FW  B000UJUJ7U  \
A2G0O4Y8QE10AE           0           0           0           0           0   
A100UD67AHFODS           5           5           5           5           0   
A105S56ODHGJEK           0           0           0           0           3   
A1075X1Q4M3S78           0           0           0           0           0   
A10C5CJK1YKGV0           0           0           0           0           0   
A10CRW7XRJBJ2G           0           0           0           0           0   
A10EIJM2C94M14           0           0           0           0           0   
A10G3LHNAK4GEH           0           0           0           0           0   
A10G4BPT5MGBHY           0           0           0           0           0   
A10GU5NVTA5I67           0           0           0           0           0   
A10NC6ZVVMBHNH           0           0           0           0           0   

asin            B0013O54OE  B001AFFZM2  B001EJQTUG  B001UO8M8M 

In [38]:
print(last_user_nn10_matrix)

asin            B005S4Y65I  B009CCVMO0  B00F8K9KZS  B00BR082FW  B000UJUJ7U  \
A2SACTIFMC5DXO           0           0           0           0           0   
A23E9QQHJLNGUI           0           0           0           0           0   
A2DV14M0BC6YY1           0           0           0           0           0   
A100UD67AHFODS           5           5           5           5           0   
A105S56ODHGJEK           0           0           0           0           3   
A1075X1Q4M3S78           0           0           0           0           0   
A10C5CJK1YKGV0           0           0           0           0           0   
A10CRW7XRJBJ2G           0           0           0           0           0   
A10EIJM2C94M14           0           0           0           0           0   
A10G3LHNAK4GEH           0           0           0           0           0   
A10G4BPT5MGBHY           0           0           0           0           0   

asin            B0013O54OE  B001AFFZM2  B001EJQTUG  B001UO8M8M 

In [39]:
fun10_np = first_user_nn10_matrix.to_numpy()
reset_fun10_np = []
for i in range(len(fun10_np)):
    reset_fun10_np.append(np.where(fun10_np[i] == 0, np.mean([item for item in fun10_np[i] if item != 0]), fun10_np[i]))
reset_fun10_cs = cosine_similarity(reset_fun10_np[1:], [reset_fun10_np[0]])
fun10_cs = cosine_similarity(fun10_np[1:],[fun10_np[0]])
lun10_np = last_user_nn10_matrix.to_numpy()
reset_lun10_np = []
for i in range(len(lun10_np)):
    reset_lun10_np.append(np.where(lun10_np[i] == 0, np.mean([item for item in lun10_np[i] if item != 0]), lun10_np[i]))
reset_lun10_cs = cosine_similarity(reset_lun10_np[1:], [reset_lun10_np[0]])
lun10_cs = cosine_similarity(lun10_np[1:],[lun10_np[0]])


In [40]:
print(fun10_cs)
print(lun10_cs)

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.2116083 ]
 [0.19762338]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]]


In [41]:
print(reset_fun10_cs)
print(reset_lun10_cs)

[[1.        ]
 [0.99779248]
 [1.        ]
 [0.99945357]
 [0.99958359]
 [1.        ]
 [0.99994213]
 [0.99931194]
 [0.99997923]
 [0.999745  ]]
[[0.9981022 ]
 [0.99836939]
 [0.99967762]
 [0.99747081]
 [0.99967762]
 [0.99913137]
 [0.99926135]
 [0.99967762]
 [0.99961978]
 [0.99898978]]


In [42]:
prediction_dic = knn_evalue.rank_k()

In [43]:
for uid in nnfor_fisrt:
    print(uid,' rr+ ',knn_evalue.rr_k_user(prediction_dic[uid],uid,len(prediction_dic[uid])))

A2G0O4Y8QE10AE  rr+  0.001594896331738437
A100UD67AHFODS  rr+  0.004608294930875576
A105S56ODHGJEK  rr+  0.002898550724637681
A1075X1Q4M3S78  rr+  0.02040816326530612
A10C5CJK1YKGV0  rr+  0.0022123893805309734
A10CRW7XRJBJ2G  rr+  0.0025575447570332483
A10EIJM2C94M14  rr+  0.0027100271002710027
A10G3LHNAK4GEH  rr+  0.02127659574468085
A10G4BPT5MGBHY  rr+  0.0024813895781637717
A10GU5NVTA5I67  rr+  0.0038314176245210726
A10NC6ZVVMBHNH  rr+  0.0013315579227696406


In [44]:
for uid in nnfor_last:
    print(uid,' rr+ ',knn_evalue.rr_k_user(prediction_dic[uid],uid,len(prediction_dic[uid])))

A2SACTIFMC5DXO  rr+  0.009433962264150943
A23E9QQHJLNGUI  rr+  0.0014814814814814814
A2DV14M0BC6YY1  rr+  0
A100UD67AHFODS  rr+  0.004608294930875576
A105S56ODHGJEK  rr+  0.002898550724637681
A1075X1Q4M3S78  rr+  0.02040816326530612
A10C5CJK1YKGV0  rr+  0.0022123893805309734
A10CRW7XRJBJ2G  rr+  0.0025575447570332483
A10EIJM2C94M14  rr+  0.0027100271002710027
A10G3LHNAK4GEH  rr+  0.02127659574468085
A10G4BPT5MGBHY  rr+  0.0024813895781637717


## Part4

In [45]:
tfidf_model = msy.Contentbased_recommendation(reuse=preprocess)
word2vec_model = msy.Contentbased_recommendation(reuse=preprocess,word2vec=True)
bert_model = msy.Contentbased_recommendation(reuse=preprocess,bert=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# bert_model = msy.Contentbased_recommendation(reuse=preprocess,bert=True)

In [46]:
tfidf_model.print_words_num()

622


In [47]:
print(tfidf_model.tfidf.to_numpy().shape)

(801, 618)


In [48]:
print('tfidf',' + ' ,cosine_similarity(tfidf_model.tfidf.iloc[:5,:].to_numpy()))
print('word2vec',' + ' ,cosine_similarity(word2vec_model.tfidf.iloc[:5,:].to_numpy()))
print('bert',' + ' ,cosine_similarity(bert_model.tfidf.iloc[:5,:].to_numpy()))

tfidf  +  [[1.         0.40876432 0.38100113 0.         0.        ]
 [0.40876432 1.         0.66379832 0.         0.        ]
 [0.38100113 0.66379832 1.         0.         0.        ]
 [0.         0.         0.         1.         0.        ]
 [0.         0.         0.         0.         1.        ]]
word2vec  +  [[1.         0.49886109 0.48628955 0.64383875 0.3779223 ]
 [0.49886109 1.         0.82148934 0.40978907 0.32713615]
 [0.48628955 0.82148934 1.         0.43663537 0.3267278 ]
 [0.64383875 0.40978907 0.43663537 1.         0.41686391]
 [0.3779223  0.32713615 0.3267278  0.41686391 1.        ]]
bert  +  [[1.         0.82374993 0.83612338 0.61856529 0.64412636]
 [0.82374993 1.         0.94925152 0.66393282 0.62677793]
 [0.83612338 0.94925152 1.         0.67859027 0.67339287]
 [0.61856529 0.66393282 0.67859027 1.         0.70500187]
 [0.64412636 0.62677793 0.67339287 0.70500187 1.        ]]


## Part 5

In [49]:
tfidf_prediction = tfidf_model.predict()
word2vec_prediction = word2vec_model.predict()
bert_prediction = bert_model.predict()

In [50]:
tfidf_evalue = msy.Evaluation(tfidf_prediction, reuse=preprocess)
word2vec_evalue = msy.Evaluation(word2vec_prediction, reuse=preprocess)
bert_evalue = msy.Evaluation(bert_prediction, reuse=preprocess)

In [51]:
print('tfidf mpk+ {:.6f}'.format(tfidf_evalue.mean_k(5,'p_k')))
print('tfidf mapk+ {:.6f}'.format(tfidf_evalue.mean_k(5,'ap_k')))
print('tfidf mrrk+ {:.6f}'.format(tfidf_evalue.mean_k(5,'rr_k')))
print('tfidf hr+ {:.6f}'.format(tfidf_evalue.mean_k(5,'hr_k')))

tfidf mpk+ 0.041613
tfidf mapk+ 0.162293
tfidf mrrk+ 0.162293
tfidf hr+ 0.208065


In [52]:
print('word2vec mpk+ {:.6f}'.format(word2vec_evalue.mean_k(5,'p_k')))
print('word2vec mapk+ {:.6f}'.format(word2vec_evalue.mean_k(5,'ap_k')))
print('word2vec mrrk+ {:.6f}'.format(word2vec_evalue.mean_k(5,'rr_k')))
print('word2vec hr+ {:.6f}'.format(word2vec_evalue.mean_k(5,'hr_k')))

word2vec mpk+ 0.035067
word2vec mapk+ 0.147078
word2vec mrrk+ 0.147078
word2vec hr+ 0.175336


In [53]:
print('bert mpk+ {:.6f}'.format(bert_evalue.mean_k(5,'p_k')))
print('bert mapk+ {:.6f}'.format(bert_evalue.mean_k(5,'ap_k')))
print('bert mrrk+ {:.6f}'.format(bert_evalue.mean_k(5,'rr_k')))
print('bert hr+ {:.6f}'.format(bert_evalue.mean_k(5,'hr_k')))

bert mpk+ 0.035885
bert mapk+ 0.061368
bert mrrk+ 0.061368
bert hr+ 0.179427


## Part 6

In [54]:
#importlib.reload(msy)

In [55]:
evaluation_list = [svd_evalue, tfidf_evalue]

In [56]:
hybrid_model = msy.Hybridrecommender_system(evaluation_list=evaluation_list, reuse=preprocess, userprofile=tfidf_model.user_mean_vector)

In [57]:
ws_pred_11 = hybrid_model.weight_strategy([1,1])
ws_pred_12 = hybrid_model.weight_strategy([1,2])
ws_pred_21 = hybrid_model.weight_strategy([2,1])

In [58]:
ws11_evalue = msy.Evaluation(ws_pred_11,reuse=preprocess)
ws12_evalue = msy.Evaluation(ws_pred_12,reuse=preprocess)
ws21_evalue = msy.Evaluation(ws_pred_21,reuse=preprocess)

In [59]:
print('ws mpk+ {:.6f} with weight [1,1]'.format(ws11_evalue.mean_k(5,'p_k')))
print('ws mapk+ {:.6f} with weight [1,1]'.format(ws11_evalue.mean_k(5,'ap_k')))
print('ws mrrk+ {:.6f} with weight [1,1]'.format(ws11_evalue.mean_k(5,'rr_k')))
print('ws hr+ {:.6f} with weight [1,1]'.format(ws11_evalue.mean_k(5,'hr_k')))

ws mpk+ 0.041847 with weight [1,1]
ws mapk+ 0.161601 with weight [1,1]
ws mrrk+ 0.161601 with weight [1,1]
ws hr+ 0.209234 with weight [1,1]


In [60]:
print('ws mpk+ {:.6f} with weight [1,2]'.format(ws12_evalue.mean_k(5,'p_k')))
print('ws mapk+ {:.6f} with weight [1,2]'.format(ws12_evalue.mean_k(5,'ap_k')))
print('ws mrrk+ {:.6f} with weight [1,2]'.format(ws12_evalue.mean_k(5,'rr_k')))
print('ws hr+ {:.6f} with weight [1,2]'.format(ws12_evalue.mean_k(5,'hr_k')))

ws mpk+ 0.042548 with weight [1,2]
ws mapk+ 0.164757 with weight [1,2]
ws mrrk+ 0.164757 with weight [1,2]
ws hr+ 0.212741 with weight [1,2]


In [61]:
print('ws mpk+ {:.6f} with weight [2,1]'.format(ws21_evalue.mean_k(5,'p_k')))
print('ws mapk+ {:.6f} with weight [2,1]'.format(ws21_evalue.mean_k(5,'ap_k')))
print('ws mrrk+ {:.6f} with weight [2,1]'.format(ws21_evalue.mean_k(5,'rr_k')))
print('ws hr+ {:.6f} with weight [2,1]'.format(ws21_evalue.mean_k(5,'hr_k')))

ws mpk+ 0.038574 with weight [2,1]
ws mapk+ 0.157423 with weight [2,1]
ws mrrk+ 0.157423 with weight [2,1]
ws hr+ 0.192870 with weight [2,1]


In [62]:
ss_pred = hybrid_model.switching_strategy()

In [63]:
ss_evalue = msy.Evaluation(ss_pred,reuse=preprocess)

In [64]:
print('ss mpk+ {:.6f}'.format(ss_evalue.mean_k(5,'p_k')))
print('ss mapk+ {:.6f}'.format(ss_evalue.mean_k(5,'ap_k')))
print('ss mrrk+ {:.6f}'.format(ss_evalue.mean_k(5,'rr_k')))
print('ss hr+ {:.6f}'.format(ss_evalue.mean_k(5,'hr_k')))

ss mpk+ 0.042314
ss mapk+ 0.164144
ss mrrk+ 0.164144
ss hr+ 0.211572


In [65]:
ms_pred = hybrid_model.metalevel_strategy(rank=5)

In [66]:
ms_evalue = msy.Evaluation(ms_pred,reuse=preprocess)

In [67]:
print('ms mpk+ {:.6f}'.format(ms_evalue.mean_k(5,'p_k')))
print('ms mapk+ {:.6f}'.format(ms_evalue.mean_k(5,'ap_k')))
print('ms mrrk+ {:.6f}'.format(ms_evalue.mean_k(5,'rr_k')))
print('ms hr+ {:.6f}'.format(ms_evalue.mean_k(5,'hr_k')))

ms mpk+ 0.011923
ms mapk+ 0.023066
ms mrrk+ 0.023066
ms hr+ 0.059614


In [68]:
evaluation_list = [knn_evalue, tfidf_evalue]

In [69]:
hybrid_model_knn = msy.Hybridrecommender_system(evaluation_list=evaluation_list, reuse=preprocess, userprofile=tfidf_model.user_mean_vector)

In [70]:
ws_pred_11_knn = hybrid_model_knn.weight_strategy([1,1])
ws11_evalue_knn = msy.Evaluation(ws_pred_11_knn,reuse=preprocess)

In [71]:
print('ws knn mpk+ {:.6f} with weight [1,1]'.format(ws11_evalue_knn.mean_k(5,'p_k')))
print('ws knn mapk+ {:.6f} with weight [1,1]'.format(ws11_evalue_knn.mean_k(5,'ap_k')))
print('ws knn mrrk+ {:.6f} with weight [1,1]'.format(ws11_evalue_knn.mean_k(5,'rr_k')))
print('ws knn hr+ {:.6f} with weight [1,1]'.format(ws11_evalue_knn.mean_k(5,'hr_k')))

ws knn mpk+ 0.008767 with weight [1,1]
ws knn mapk+ 0.021420 with weight [1,1]
ws knn mrrk+ 0.021420 with weight [1,1]
ws knn hr+ 0.043834 with weight [1,1]


In [72]:
bf_model = hybrid_model.weight_strategy(borda=True)

In [73]:
bf_evalue = msy.Evaluation(bf_model, reuse=preprocess)

In [74]:
print('bf mpk+ {:.6f}'.format(bf_evalue.mean_k(5,'p_k')))
print('bf mapk+ {:.6f}'.format(bf_evalue.mean_k(5,'ap_k')))
print('bf mrrk+ {:.6f}'.format(bf_evalue.mean_k(5,'rr_k')))
print('bf hr+ {:.6f}'.format(bf_evalue.mean_k(5,'hr_k')))

bf mpk+ 0.032262
bf mapk+ 0.076476
bf mrrk+ 0.076476
bf hr+ 0.161309
